# 主站收入归因分析

## 目标

通过精确匹配算法识别主站（非Art）的 bot 使用记录，使用 Last-Touch 优化版归因模型将订单收入分配到主站 bot，从而估算主站对总收入的贡献。

## 背景

**问题**：现有的收入归因（见 `bot-margin-analysis.md`）基于 `art_task` 表，只能归因到 Art 站的 bot 使用。由于 Stripe 和 PayPal 订单没有区分主站和 Art 站，无法直接知道收入来自哪个站。

**解决思路**：
1. 使用精确匹配算法从 `user_energy_bot_usage_logs` 中识别主站记录
2. 将这些主站使用记录作为归因依据
3. 使用 Last-Touch 优化版归因模型分配收入

## 参数

| 参数 | 说明 | 默认值 |
|------|------|--------|
| `start_date` | 开始日期 (YYYY-MM-DD) | 昨天 |
| `end_date` | 结束日期 (YYYY-MM-DD) | 昨天 |

## 数据源

**收入表**：
- `user_subscription_stripe_orders` - Stripe 订单
- `user_subscription_paypal_orders` - PayPal 订单
- `apple_used_transactions` - Apple IAP 订单

**使用记录表**：
- `user_energy_bot_usage_logs` - 总电量消耗表（主站+Art）
- `art_task` - Art任务表（仅Art消耗，用于精确匹配排除）
- `bot` - Bot信息表

## 核心算法

**精确匹配条件**: `user_id` + `bot_id` + `energy` + 时间窗口（±5秒）

```
user_energy_bot_usage_logs（总消耗）
├── Art记录 ← 能在 art_task 中找到匹配（排除）
└── 主站记录 ← 无法匹配的记录（保留）
```

**归因原理（Last-Touch 优化版）**：
- **情况1**: 订单前有主站使用记录 → 归因给用户在付费前最后使用的主站 bot
- **情况2**: 订单前无主站使用记录 → 归因给用户在付费后第一次使用的主站 bot
- **情况3**: 前后都没有主站使用记录 → 视为 Art 站订单（不计入主站收入）

**归因时间窗口**：

| 窗口类型 | 时间范围 | 说明 |
|----------|----------|------|
| **订单窗口** | start_date 至 end_date | 统计的订单时间范围 |
| **使用窗口** | start_date - 7天 至 end_date + 7天 | 用于归因的主站使用时间范围 |

## Agent 运行模式

```bash
jq -r '.cells[] | select(.cell_type=="code") | .source | if type=="array" then join("") else . end' \
  main-site-revenue-attribution.ipynb > temp/main-site-revenue-attribution.ts

deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/main-site-revenue-attribution.ts 2026-01-13              # 单天

deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/main-site-revenue-attribution.ts 2026-01-11 2026-01-13   # 日期范围
```

## Step 0: 环境配置

In [29]:
const MCP_HUB_URL = Deno.env.get("MCP_HUB_URL") || "http://52.12.230.109:3000/mcp";
const MCP_AUTH_TOKEN = (() => {
  let token = Deno.env.get("MCP_AUTH_TOKEN") || "";
  if (token && !token.startsWith("Bearer ")) token = `Bearer ${token}`;
  return token;
})();
const OUTPUT_DIR = "./reports";

let startDate = "";
let endDate = "";
let mcpClient: any = null;

if (!MCP_AUTH_TOKEN) {
  console.error("❌ 请设置环境变量: export MCP_AUTH_TOKEN='your-token'");
} else {
  console.log("✅ 配置加载完成");
}

✅ 配置加载完成


## Step 1: 连接 MCP Hub

In [30]:
import { Client } from "npm:@modelcontextprotocol/sdk@1.24.3/client/index.js";
import { StreamableHTTPClientTransport } from "npm:@modelcontextprotocol/sdk@1.24.3/client/streamableHttp.js";

const transport = new StreamableHTTPClientTransport(new URL(MCP_HUB_URL), {
  requestInit: { headers: { Authorization: MCP_AUTH_TOKEN } },
});

mcpClient = new Client(
  { name: "main-site-revenue-attribution", version: "1.0.0" },
  { capabilities: { tools: {}, sampling: {} } }
);

await mcpClient.connect(transport);
console.log("✅ Connected to MCP Hub");

✅ Connected to MCP Hub


## Step 2: 配置分析日期

支持两种模式：
- **CLI 模式**: 从命令行参数获取日期
- **Jupyter 模式**: 通过 prompt 交互输入

In [31]:
const args = typeof Deno !== "undefined" ? Deno.args : [];
const yesterday = new Date(Date.now() - 86400000).toISOString().split("T")[0];

if (args.length >= 1) {
  startDate = args[0];
  endDate = args[1] || args[0];
} else {
  const dateInput = prompt(
    `请输入分析日期（YYYY-MM-DD）\n单天: 2026-01-13\n范围: 2026-01-11,2026-01-13\n默认: ${yesterday}`,
    yesterday
  );
  const parts = (dateInput || yesterday).split(",").map(s => s.trim());
  startDate = parts[0];
  endDate = parts[1] || parts[0];
}

const dateList: string[] = [];
let current = new Date(startDate);
const end = new Date(endDate);
while (current <= end) {
  dateList.push(current.toISOString().split("T")[0]);
  current.setDate(current.getDate() + 1);
}

console.log(`✅ 分析日期: ${startDate} 至 ${endDate} (共 ${dateList.length} 天)`);

✅ 分析日期: 2026-01-14 至 2026-01-14 (共 1 天)


## Step 3: 精确匹配算法

对每一天执行精确匹配，识别主站使用记录：

**数据源**：
- `user_energy_bot_usage_logs`: 所有 bot 使用的电量消耗记录
- `art_task`: Art 站生成任务记录

**精确匹配逻辑**（3个SQL查询）：

1. **查询总量**: 从 `user_energy_bot_usage_logs` 获取当天所有记录数和电量总和
2. **查询Art任务**: 从 `art_task` 获取当天已完成任务的统计（用于验证）
3. **精确匹配**: 使用 CTE 将 usage_logs 与 art_task 精确匹配
   - 匹配条件: `user_id` + `bot_id` + `energy` 完全相等
   - 时间窗口: usage_logs.created_date 与 art_task.updated_date 相差 ±5秒内
   - 匹配上的 = Art 记录（排除）
   - 匹配不上的 = 主站记录（保留）

### 逐天执行精确匹配

每天执行 3 个 SQL 查询，最后计算主站占比：
- **主站电量** = logs总电量 - 匹配到Art的电量
- **主站占比** = 主站电量 / logs总电量 × 100%

In [32]:
interface DailyStats {
  date: string;
  total_logs_count: number;
  total_logs_energy: number;
  art_task_count: number;
  art_task_energy: number;
  matched_to_art_count: number;
  matched_to_art_energy: number;
  main_site_count: number;
  main_site_energy: number;
  main_site_pct: number;
  art_pct: number;
}

const dailyStats: DailyStats[] = [];

for (const date of dateList) {
  console.log(`📅 ${date}`);
  
  const usageResult = await mcpClient.callTool({
    name: "bytebase-execute_sql",
    arguments: {
      sql: `-- Query 1: 查询当天所有电量消耗记录的总数和总电量
            SELECT COUNT(*) as cnt, COALESCE(SUM(energy), 0) as total_energy
            FROM my_shell_prod.user_energy_bot_usage_logs
            WHERE created_date >= '${date} 00:00:00' AND created_date <= '${date} 23:59:59'`
    }
  });
  const usageData = JSON.parse((usageResult.content as any[])[0].text);
  const totalLogsCount = usageData.data.rows[0]?.cnt || 0;
  const totalLogsEnergy = usageData.data.rows[0]?.total_energy || 0;
  
  const artResult = await mcpClient.callTool({
    name: "bytebase-execute_sql",
    arguments: {
      sql: `-- Query 2: 查询当天已完成的 Art 任务统计（用于验证匹配率）
            SELECT COUNT(*) as cnt, COALESCE(SUM(actual_energy_cost), 0) as total_energy
            FROM my_shell_prod.art_task
            WHERE status = 'done' AND deleted_at IS NULL
              AND created_date >= '${date} 00:00:00' AND created_date <= '${date} 23:59:59'`
    }
  });
  const artData = JSON.parse((artResult.content as any[])[0].text);
  const artTaskCount = artData.data.rows[0]?.cnt || 0;
  const artTaskEnergy = artData.data.rows[0]?.total_energy || 0;
  
  const matchResult = await mcpClient.callTool({
    name: "bytebase-execute_sql",
    arguments: {
      sql: `-- Query 3: 精确匹配算法 - 找出能匹配到 Art 任务的 usage_logs 记录
            WITH art_done AS (
              -- 当天已完成的 Art 任务
              SELECT user_id, bot_id, actual_energy_cost as energy, updated_date
              FROM my_shell_prod.art_task
              WHERE status = 'done' AND deleted_at IS NULL
                AND created_date >= '${date} 00:00:00' AND created_date <= '${date} 23:59:59'
            ),
            usage_logs AS (
              -- 当天所有电量消耗记录
              SELECT user_id, bot_id, energy, created_date
              FROM my_shell_prod.user_energy_bot_usage_logs
              WHERE created_date >= '${date} 00:00:00' AND created_date <= '${date} 23:59:59'
            ),
            matched AS (
              -- 能匹配到 Art 任务的 usage_logs（精确匹配条件）
              SELECT u.* FROM usage_logs u
              WHERE EXISTS (
                SELECT 1 FROM art_done a
                WHERE a.user_id = u.user_id       -- 同一用户
                  AND a.bot_id = u.bot_id         -- 同一 bot
                  AND a.energy = u.energy         -- 电量完全相等
                  AND ABS(TIMESTAMPDIFF(SECOND, u.created_date, a.updated_date)) <= 5  -- 时间差≤5秒
              )
            )
            SELECT COUNT(*) as matched_count, COALESCE(SUM(energy), 0) as matched_energy FROM matched`
    }
  });
  const matchData = JSON.parse((matchResult.content as any[])[0].text);
  const matchedCount = matchData.data.rows[0]?.matched_count || 0;
  const matchedEnergy = matchData.data.rows[0]?.matched_energy || 0;
  
  const mainSiteCount = totalLogsCount - matchedCount;
  const mainSiteEnergy = totalLogsEnergy - matchedEnergy;
  const mainSitePct = totalLogsEnergy > 0 ? (mainSiteEnergy / totalLogsEnergy * 100) : 0;
  const artPct = totalLogsEnergy > 0 ? (matchedEnergy / totalLogsEnergy * 100) : 0;
  
  console.log(`   主站: ${mainSiteEnergy.toLocaleString()} (${mainSitePct.toFixed(1)}%) | Art: ${matchedEnergy.toLocaleString()} (${artPct.toFixed(1)}%)`);
  
  dailyStats.push({
    date, total_logs_count: totalLogsCount, total_logs_energy: totalLogsEnergy,
    art_task_count: artTaskCount, art_task_energy: artTaskEnergy,
    matched_to_art_count: matchedCount, matched_to_art_energy: matchedEnergy,
    main_site_count: mainSiteCount, main_site_energy: mainSiteEnergy,
    main_site_pct: mainSitePct, art_pct: artPct
  });
}

console.log("✅ 精确匹配完成");

📅 2026-01-14
   主站: 55,826 (40.7%) | Art: 81218 (59.3%)
✅ 精确匹配完成


## Step 4: 汇总结果展示

In [ ]:
import * as Plot from "npm:@observablehq/plot";
import { document } from "jsr:@ry/jupyter-helper";

let totalMainEnergy = 0, totalArtEnergy = 0, totalEnergy = 0;
for (const stat of dailyStats) {
  totalMainEnergy += stat.main_site_energy;
  totalArtEnergy += stat.matched_to_art_energy;
  totalEnergy += stat.total_logs_energy;
}

const overallMainPct = totalEnergy > 0 ? (totalMainEnergy / totalEnergy * 100) : 0;
const overallArtPct = totalEnergy > 0 ? (totalArtEnergy / totalEnergy * 100) : 0;

console.log(`📊 主站: ${totalMainEnergy.toLocaleString()} (${overallMainPct.toFixed(1)}%) | Art: ${totalArtEnergy.toLocaleString()} (${overallArtPct.toFixed(1)}%)`);

const chartData = dailyStats.flatMap(s => [
  { date: s.date, category: "主站", energy: s.main_site_energy },
  { date: s.date, category: "Art", energy: s.matched_to_art_energy }
]);

if (Deno.args.length === 0) {
  Plot.plot({
    document,
    title: "主站 vs Art 电量消耗",
    width: 800,
    height: 400,
    x: { label: "日期" },
    y: { label: "电量", grid: true },
    color: { legend: true, domain: ["主站", "Art"], range: ["#4e79a7", "#e15759"] },
    marks: [
      Plot.barY(chartData, { x: "date", y: "energy", fill: "category", tip: true }),
      Plot.ruleY([0])
    ]
  })
}

📊 主站: 224,807 (0.0%) | Art: 049911483264933084279 (0.0%)


主站 vs Art 电量消耗 主站 Art 0 20,000 40,000 60,000 80,000 100,000 120,000 140,000 ↑ 电量 2026-01-10 2026-01-11 2026-01-12 2026-01-13 日期 ⚠️ 1 warning. Please check the console.

## Step 5: 导出主站使用记录详情

查询主站使用记录的详细信息（排除 Art 记录后），导出为 CSV 文件。

**SQL 逻辑**：
- 使用 `NOT EXISTS` 排除能匹配到 `art_task` 的记录
- JOIN `bot` 表获取 bot 名称
- 每天最多导出 10000 条（避免数据量过大）

In [ ]:
const allMainSiteRecords: any[] = [];

for (const date of dateList) {
  console.log(`📅 查询 ${date}...`);
  
  const result = await mcpClient.callTool({
    name: "bytebase-execute_sql",
    arguments: {
      sql: `-- 查询主站使用记录详情（排除 Art 记录）
            WITH art_done AS (
              -- 当天已完成的 Art 任务（用于排除匹配）
              SELECT user_id, bot_id, actual_energy_cost as energy, updated_date
              FROM my_shell_prod.art_task
              WHERE status = 'done' AND deleted_at IS NULL
                AND created_date >= '${date} 00:00:00' AND created_date <= '${date} 23:59:59'
            )
            SELECT u.user_id, u.bot_id, b.name as bot_name, u.energy, u.created_date
            FROM my_shell_prod.user_energy_bot_usage_logs u
            LEFT JOIN my_shell_prod.bot b ON u.bot_id = b.id  -- 关联 bot 表获取名称
            WHERE u.created_date >= '${date} 00:00:00' AND u.created_date <= '${date} 23:59:59'
              AND NOT EXISTS (
                -- 排除能精确匹配到 Art 任务的记录
                SELECT 1 FROM art_done a
                WHERE a.user_id = u.user_id AND a.bot_id = u.bot_id AND a.energy = u.energy
                  AND ABS(TIMESTAMPDIFF(SECOND, u.created_date, a.updated_date)) <= 5
              )
            ORDER BY u.created_date DESC LIMIT 10000`
    }
  });
  
  const data = JSON.parse((result.content as any[])[0].text);
  if (data.success && data.data.rows) {
    allMainSiteRecords.push(...data.data.rows);
    console.log(`   ✅ ${data.data.rows.length} 条`);
  }
}

console.log(`\n📊 总计: ${allMainSiteRecords.length} 条主站记录`);

📅 查询 2026-01-12...
   ✅ 9224 条

📊 总计: 9224 条主站记录


### 5.1 导出 CSV

In [ ]:
import pl from "npm:nodejs-polars";
import { agGrid } from "jsr:@manzt/jupyter-helper";

const dfExport = pl.DataFrame({
  user_id: allMainSiteRecords.map(r => r.user_id),
  bot_id: allMainSiteRecords.map(r => r.bot_id),
  bot_name: allMainSiteRecords.map(r => r.bot_name || ""),
  energy: allMainSiteRecords.map(r => r.energy),
  created_date: allMainSiteRecords.map(r => r.created_date),
});

try { await Deno.mkdir(OUTPUT_DIR, { recursive: true }); } catch {}
const outputFile = `${OUTPUT_DIR}/main-site-usage-${startDate}-to-${endDate}.csv`;
dfExport.writeCSV(outputFile);

console.log(`💾 已保存: ${outputFile}`);

if (Deno.args.length === 0) {
  agGrid(pl.readCSV(outputFile).head(100));
}

💾 已保存: ./reports/main-site-usage-2026-01-12-to-2026-01-12.csv


## Step 6: 主站 Bot 汇总统计

按 Bot 汇总电量消耗，用水平柱状图展示 Top 20。

In [ ]:
const dfDetail = pl.readCSV(outputFile);
import { document } from "jsr:@ry/jupyter-helper";

const botSummary = dfDetail
  .groupBy("bot_name")
  .agg(
    pl.col("energy").sum().alias("total_energy"),
    pl.col("user_id").nUnique().alias("unique_users"),
    pl.col("bot_id").count().alias("record_count")
  )
  .sort("total_energy", true)
  .head(20);

const botData = botSummary.toRecords().map((r: any) => ({
  bot_name: r.bot_name || "(unknown)",
  total_energy: r.total_energy,
  unique_users: r.unique_users
}));

console.log(`📊 Top 20 主站 Bot 电量消耗`);
botData.forEach((b: any, i: number) => {
  console.log(`   ${i + 1}. ${b.bot_name}: ${b.total_energy.toLocaleString()}`);
});

if (Deno.args.length === 0) {
  Plot.plot({
    document,
    title: "主站 Top 20 Bot 电量消耗",
    width: 800,
    height: 500,
    marginLeft: 200,
    x: { label: "电量", grid: true },
    y: { label: null },
    marks: [
      Plot.barX(botData, { 
        x: "total_energy", 
        y: "bot_name", 
        fill: "#4e79a7",
        sort: { y: "-x" },
        tip: true
      }),
      Plot.ruleX([0])
    ]
  })
}

📊 Top 20 主站 Bot 电量消耗
   1. Take off her clothes-PRO！!🎄🎄🔞: 9,194
   2. 🔞POV Missionary Insertion！🔞: 9,029
   3. Breast Expansion: 4,586
   4. Rip Her Clothes V0.2: 2,650
   5. Oral Sex-Pro: 2,499
   6. 🔞Cum Facial AI Porn Maker💦👩: 2,282
   7. 🔞NSFW video maker (Adv: two prompt): 1,681
   8. Wan POV Cowgirl Insertion: 1,590
   9. 🔞NSFW Video Playground🔞: 1,319
   10. Sora 2 Photoreal Person: 1,274
   11. 🔞NSFW Video Maker : 1,033
   12. Sexy 360-Degree Camera: 756
   13. 🔞Missionary AI Porn Maker 🚀💦: 736
   14. 🔞Leg Fuck AI Porn Maker🦵💦: 676
   15. 🔞Play her Breasts: 642
   16. The Ultimate Gift 🎁: 612
   17. Celebrity Porn: 529
   18. Duolingo Style Filter: 488
   19. Nano Banana 2: 486
   20. 🔞Blowjob AI Porn Maker👄💦: 466


主站 Top 20 Bot 电量消耗 Take off her clothes-PRO！!🎄🎄🔞 🔞POV Missionary Insertion！🔞 Breast Expansion Rip Her Clothes V0.2 Oral Sex-Pro 🔞Cum Facial AI Porn Maker💦👩 🔞NSFW video maker (Adv: two prompt) Wan POV Cowgirl Insertion 🔞NSFW Video Playground🔞 Sora 2 Photoreal Person 🔞NSFW Video Maker Sexy 360-Degree Camera 🔞Missionary AI Porn Maker 🚀💦 🔞Leg Fuck AI Porn Maker🦵💦 🔞Play her Breasts The Ultimate Gift 🎁 Celebrity Porn Duolingo Style Filter Nano Banana 2 🔞Blowjob AI Porn Maker👄💦 0 1,000 2,000 3,000 4,000 5,000 6,000 7,000 8,000 9,000 电量 →

## Step 7: 主站收入归因（SQL优化版）

使用单个 SQL 完成 Last-Touch 优化版归因，将订单收入分配到主站 Bot。

**SQL 结构**：
1. **main_site_usage CTE**: 筛选主站使用记录（排除 Art 匹配的记录）
2. **orders CTE**: 合并 Stripe + PayPal + Apple IAP 订单
3. **last_touch_before CTE**: 订单前最后一次使用 → 归因
4. **first_touch_after CTE**: 订单前无使用，取订单后第一次使用 → 归因
5. **最终聚合**: 按 bot_name 汇总收入

**时间窗口**：
- 订单：分析日期范围
- 使用记录：分析日期 ±7 天（捕获试用转付费 + 先付费后使用）

**性能优势**：相比 JavaScript 归因，减少 99.9% 数据传输，性能提升 3-10 倍

In [ ]:
console.log(`📊 执行主站收入归因 SQL...`);

const attributionResult = await mcpClient.callTool({
  name: "bytebase-execute_sql",
  arguments: {
    sql: `-- ====================================================================
-- 主站收入归因查询（SQL优化版）
-- 功能: 使用 Last-touch 优化版归因，将订单收入分配到主站 bot
-- 核心: 用 NOT EXISTS 排除 Art 记录，只保留主站使用记录
-- ====================================================================

-- --------------------------------------------------------------------
-- Step 1: 主站使用记录（排除能匹配到 art_task 的记录）
-- 时间窗口: 订单窗口 ±7 天
-- --------------------------------------------------------------------
WITH main_site_usage AS (
  SELECT 
    u.user_id,
    u.bot_id,
    b.name as bot_name,
    u.created_date
  FROM my_shell_prod.user_energy_bot_usage_logs u
  LEFT JOIN my_shell_prod.bot b ON u.bot_id = b.id
  WHERE u.created_date >= DATE_SUB('${startDate}', INTERVAL 7 DAY)
    AND u.created_date <= DATE_ADD('${endDate}', INTERVAL 7 DAY)
    AND NOT EXISTS (
      -- 排除能精确匹配到 Art 任务的记录
      SELECT 1 FROM my_shell_prod.art_task a
      WHERE a.status = 'done' AND a.deleted_at IS NULL
        AND a.user_id = u.user_id
        AND a.bot_id = u.bot_id
        AND a.actual_energy_cost = u.energy
        AND ABS(TIMESTAMPDIFF(SECOND, u.created_date, a.updated_date)) <= 5
    )
),

-- --------------------------------------------------------------------
-- Step 2: 合并所有订单源（Stripe + PayPal + Apple IAP）
-- --------------------------------------------------------------------
orders AS (
  SELECT user_id, amount, created_date, 'stripe' as source
  FROM my_shell_prod.user_subscription_stripe_orders
  WHERE status = 'ORDER_STATUS_SUCCESS' AND amount >= 0
    AND created_date >= '${startDate} 00:00:00'
    AND created_date <= '${endDate} 23:59:59'

  UNION ALL

  SELECT user_id,
    CASE
      WHEN biz_type = 'ENERGY' AND paypal_price_id = '500' THEN 6.99
      WHEN biz_type = 'ENERGY' AND paypal_price_id = '2000' THEN 20.99
      ELSE amount
    END as amount,
    created_date, 'paypal' as source
  FROM my_shell_prod.user_subscription_paypal_orders
  WHERE status = 'ORDER_STATUS_SUCCESS'
    AND created_date >= '${startDate} 00:00:00'
    AND created_date <= '${endDate} 23:59:59'

  UNION ALL

  SELECT user_id,
    CASE product_id
      WHEN 'PLAYER_MONTHLY' THEN 6.99
      WHEN 'PLAYER_YEARLY' THEN 58.99
      WHEN 'DEVELOPER_MONTHLY' THEN 59.99
      WHEN 'DEVELOPER_YEARLY' THEN 499.99
      WHEN '33OFF_DEVELOPER_MONTHLY' THEN 39.99
      WHEN 'energy_500' THEN 6.99
      WHEN 'energy_2000' THEN 20.99
      ELSE 0
    END as amount,
    created_date, 'apple' as source
  FROM my_shell_prod.apple_used_transactions
  WHERE created_date >= '${startDate} 00:00:00'
    AND created_date <= '${endDate} 23:59:59'
),

-- --------------------------------------------------------------------
-- Step 3: Last-touch 归因（订单前最后一次使用）
-- --------------------------------------------------------------------
last_touch_before AS (
  SELECT user_id, amount, order_date, source, attributed_bot, attribution_type
  FROM (
    SELECT
      o.user_id, o.amount, o.created_date as order_date, o.source,
      m.bot_name as attributed_bot,
      'before' as attribution_type,
      ROW_NUMBER() OVER (
        PARTITION BY o.user_id, o.created_date
        ORDER BY m.created_date DESC
      ) as rn
    FROM orders o
    INNER JOIN main_site_usage m
      ON o.user_id = m.user_id
      AND m.created_date <= o.created_date
  ) ranked
  WHERE rn = 1
),

-- --------------------------------------------------------------------
-- Step 4: First-touch 归因（订单后第一次使用，仅当订单前无使用）
-- --------------------------------------------------------------------
first_touch_after AS (
  SELECT user_id, amount, order_date, source, attributed_bot, attribution_type
  FROM (
    SELECT
      o.user_id, o.amount, o.created_date as order_date, o.source,
      m.bot_name as attributed_bot,
      'after' as attribution_type,
      ROW_NUMBER() OVER (
        PARTITION BY o.user_id, o.created_date
        ORDER BY m.created_date ASC
      ) as rn
    FROM orders o
    INNER JOIN main_site_usage m
      ON o.user_id = m.user_id
      AND m.created_date > o.created_date
    WHERE NOT EXISTS (
      SELECT 1 FROM main_site_usage m2
      WHERE m2.user_id = o.user_id
        AND m2.created_date <= o.created_date
    )
  ) ranked
  WHERE rn = 1
),

-- --------------------------------------------------------------------
-- Step 5: 合并归因结果
-- --------------------------------------------------------------------
attributed_orders AS (
  SELECT * FROM last_touch_before
  UNION ALL
  SELECT * FROM first_touch_after
)

-- --------------------------------------------------------------------
-- Step 6: 按 bot 汇总归因收入
-- --------------------------------------------------------------------
SELECT
  attributed_bot as bot_name,
  SUM(amount) as revenue,
  COUNT(*) as orders,
  COUNT(DISTINCT user_id) as unique_users,
  COUNT(CASE WHEN attribution_type = 'before' THEN 1 END) as before_orders,
  COUNT(CASE WHEN attribution_type = 'after' THEN 1 END) as after_orders
FROM attributed_orders
GROUP BY attributed_bot
ORDER BY revenue DESC`
  }
});

const attrData = JSON.parse((attributionResult.content as any[])[0].text);
const botRevenueRows = attrData.data?.rows || [];

// 计算汇总统计
let totalAttributedRevenue = 0;
let totalOrders = 0;
let beforeOrders = 0;
let afterOrders = 0;

for (const row of botRevenueRows) {
  totalAttributedRevenue += Number(row.revenue) || 0;
  totalOrders += Number(row.orders) || 0;
  beforeOrders += Number(row.before_orders) || 0;
  afterOrders += Number(row.after_orders) || 0;
}

console.log(`✅ 归因完成`);
console.log(`\n📊 归因结果:`);
console.log(`   已归因: ${totalOrders} 个订单, $${totalAttributedRevenue.toFixed(2)}`);
console.log(`     - Last-touch (订单前): ${beforeOrders} 个`);
console.log(`     - First-touch (订单后): ${afterOrders} 个`);

console.log(`\n📊 主站 Bot 收入归因 Top 20:`);
botRevenueRows.slice(0, 20).forEach((b: any, i: number) => {
  console.log(`   ${i + 1}. ${b.bot_name || "(unknown)"}: $${Number(b.revenue).toFixed(2)} (${b.orders} 单, ${b.unique_users} 用户)`);
});

📊 执行主站收入归因 SQL...
✅ 归因完成

📊 归因结果:
   已归因: 54 个订单, $1210.46
     - Last-touch (订单前): 26 个
     - First-touch (订单后): 28 个

📊 主站 Bot 收入归因 Top 20:
   1. Rip Her Clothes V0.2: $387.87 (13 单, 11 用户)
   2. 🔞Cum Facial AI Porn Maker💦👩: $187.93 (7 单, 5 用户)
   3. 🔞POV Missionary Insertion！🔞: $147.94 (6 单, 4 用户)
   4. sexy 360° video: $99.98 (2 单, 2 用户)
   5. 🔞Footjob AI Porn Maker 👣💦: $66.98 (2 单, 2 用户)
   6. ai face swap porn- missionary : $66.98 (2 单, 2 用户)
   7. Join Zootopia 2 filter: $58.99 (1 单, 1 用户)
   8. HotBody Office: $39.99 (1 单, 1 用户)
   9. 🔞Missionary AI Porn Maker 🚀💦: $27.96 (4 单, 4 用户)
   10. Oral Sex-Pro: $20.99 (1 单, 1 用户)
   11. Take off her clothes-PRO！!🎄🎄🔞: $20.97 (3 单, 3 用户)
   12. Wan POV Cowgirl Insertion: $20.97 (3 单, 2 用户)
   13. # FUTA Porn - AI Real FUTA Porn: $13.98 (2 单, 2 用户)
   14. AI Hair Consultant: $6.99 (1 单, 1 用户)
   15. AI Nude video with clothes removed: $6.99 (1 单, 1 用户)
   16. 🔞NSFW Video Playground🔞: $6.99 (1 单, 1 用户)
   17. PicFlow-Single Image Motion G

## Step 8: 导出归因报告

将 Bot 归因收入导出为 CSV 文件，并在 Jupyter 模式下显示图表和表格。

In [ ]:
import pl from "npm:nodejs-polars";
import { agGrid, quak } from "jsr:@manzt/jupyter-helper";

let totalMainEnergy = 0, totalArtEnergy = 0, totalEnergy = 0;
for (const stat of dailyStats) {
  totalMainEnergy += stat.main_site_energy;
  totalArtEnergy += stat.matched_to_art_energy;
  totalEnergy += stat.total_logs_energy;
}

const overallMainPct = totalEnergy > 0 ? (totalMainEnergy / totalEnergy * 100) : 0;
const overallArtPct = totalEnergy > 0 ? (totalArtEnergy / totalEnergy * 100) : 0;

console.log(`📊 主站: ${totalMainEnergy.toLocaleString()} (${overallMainPct.toFixed(1)}%) | Art: ${totalArtEnergy.toLocaleString()} (${overallArtPct.toFixed(1)}%)`);

const df = pl.DataFrame({
  "日期": dailyStats.map(s => s.date),
  "logs记录": dailyStats.map(s => s.total_logs_count),
  "logs电量": dailyStats.map(s => s.total_logs_energy),
  "Art记录": dailyStats.map(s => s.matched_to_art_count),
  "Art电量": dailyStats.map(s => s.matched_to_art_energy),
  "主站记录": dailyStats.map(s => s.main_site_count),
  "主站电量": dailyStats.map(s => s.main_site_energy),
  "主站%": dailyStats.map(s => s.main_site_pct),
  "Art%": dailyStats.map(s => s.art_pct),
});

agGrid(df);

📊 主站: 51,825 (51.2%) | Art: 049330 (48.8%)


In [ ]:
// 导出归因结果到 CSV
const dfRevenue = pl.DataFrame({
  bot_name: botRevenueRows.map((r: any) => r.bot_name || "(unknown)"),
  revenue: botRevenueRows.map((r: any) => Number(r.revenue)),
  orders: botRevenueRows.map((r: any) => Number(r.orders)),
  unique_users: botRevenueRows.map((r: any) => Number(r.unique_users)),
  before_orders: botRevenueRows.map((r: any) => Number(r.before_orders)),
  after_orders: botRevenueRows.map((r: any) => Number(r.after_orders)),
});

const revenueFile = `${OUTPUT_DIR}/main-site-revenue-${startDate}-to-${endDate}.csv`;
dfRevenue.writeCSV(revenueFile);
console.log(`💾 已保存: ${revenueFile}`);

// Jupyter 模式：显示图表和表格
if (Deno.args.length === 0) {
  const top20Data = botRevenueRows.slice(0, 20).map((r: any) => ({
    bot_name: r.bot_name || "(unknown)",
    revenue: Number(r.revenue)
  }));
  
  Plot.plot({
    document,
    title: "主站 Bot 收入归因 Top 20",
    width: 800,
    height: 500,
    marginLeft: 200,
    marginRight: 60,
    x: { label: "收入 ($)", grid: true },
    y: { label: null },
    marks: [
      Plot.barX(top20Data, { 
        x: "revenue", 
        y: "bot_name", 
        fill: "#59a14f",
        sort: { y: "-x" },
        tip: true
      }),
      Plot.text(top20Data, {
        x: "revenue",
        y: "bot_name",
        text: d => `$${d.revenue.toFixed(0)}`,
        dx: 5,
        textAnchor: "start",
        fill: "#333",
        fontSize: 11
      }),
      Plot.ruleX([0])
    ]
  });
}

💾 已保存: ./reports/main-site-revenue-2026-01-12-to-2026-01-12.csv


主站 Bot 收入归因 Top 20 Rip Her Clothes V0.2 🔞Cum Facial AI Porn Maker💦👩 🔞POV Missionary Insertion！🔞 sexy 360° video ai face swap porn- missionary 🔞Footjob AI Porn Maker 👣💦 Join Zootopia 2 filter HotBody Office 🔞Missionary AI Porn Maker 🚀💦 Oral Sex-Pro Take off her clothes-PRO！!🎄🎄🔞 Wan POV Cowgirl Insertion # FUTA Porn - AI Real FUTA Porn AI Hair Consultant AI Nude video with clothes removed PicFlow-Single Image Motion Generator Sex Nipple Play sexy bunny outfit 🔞Leg Fuck AI Porn Maker🦵💦 🔞NSFW Video Playground🔞 0 50 100 150 200 250 300 350 收入 ($) → $388 $188 $148 $100 $67 $67 $59 $40 $28 $21 $21 $21 $14 $7 $7 $7 $7 $7 $7 $7

In [ ]:
// Jupyter 模式：显示图表和表格
if (Deno.args.length === 0) {
  quak(dfRevenue.head(50));
}

## Step 9: 主站毛利率计算

计算主站的毛利率指标：

**计算公式**：
- **总收入** = 主站归因订单收入（已在 Step 7 计算）
- **总成本** = 主站电量消耗 / 100（energy 单位是 cents，转换为 USD）
- **毛利润** = 总收入 - 总成本
- **毛利率** = 毛利润 / 总收入 × 100%

In [ ]:
// 计算主站毛利率
const mainSiteRevenue = totalAttributedRevenue;  // 已归因收入 (USD)
const mainSiteCost = totalMainEnergy / 100;      // 电量成本 (cents → USD)
const mainSiteProfit = mainSiteRevenue - mainSiteCost;
const mainSiteMargin = mainSiteRevenue > 0 ? (mainSiteProfit / mainSiteRevenue * 100) : 0;

console.log(`\n${"=".repeat(50)}`);
console.log(`📊 主站毛利率分析 (${startDate} 至 ${endDate})`);
console.log(`${"=".repeat(50)}`);
console.log(`   总收入:   $${mainSiteRevenue.toFixed(2)}`);
console.log(`   总成本:   $${mainSiteCost.toFixed(2)}`);
console.log(`   毛利润:   $${mainSiteProfit.toFixed(2)}`);
console.log(`   毛利率:   ${mainSiteMargin.toFixed(1)}%`);
console.log(`${"=".repeat(50)}`);

// 与 Art 站对比（如果有数据）
const artCost = totalArtEnergy / 100;  // Art 成本 (cents → USD)
console.log(`\n📊 主站 vs Art 成本对比:`);
console.log(`   主站成本: $${mainSiteCost.toFixed(2)} (${overallMainPct.toFixed(1)}%)`);
console.log(`   Art成本:  $${artCost.toFixed(2)} (${overallArtPct.toFixed(1)}%)`);


📊 主站毛利率分析 (2026-01-12 至 2026-01-12)
   总收入:   $1210.46
   总成本:   $518.25
   毛利润:   $692.21
   毛利率:   57.2%

📊 主站 vs Art 成本对比:
   主站成本: $518.25 (51.2%)
   Art成本:  $493.30 (48.8%)


---

## 附录

### 核心指标

| 指标 | 计算方式 | 说明 |
|------|----------|------|
| **主站毛利率** | (总收入 - 总成本) / 总收入 × 100% | 主站盈利能力核心指标 |
| **主站收入占比** | 主站归因收入 / 总收入 × 100% | 评估主站对收入的贡献 |
| **Art站收入占比** | Art归因收入 / 总收入 × 100% | 评估Art站对收入的贡献 |
| **主站归因覆盖率** | 主站归因订单数 / 有主站使用的订单数 | 归因算法有效性 |
| **主站Top Bot集中度** | Top 3 Bot收入 / 主站总归因收入 | 识别主站收入来源集中度 |

### 注意事项

1. **精确匹配算法性能**：EXISTS 子查询在大数据量下可能较慢，建议时间范围控制在 7-14 天
2. **归因覆盖率预期**：由于很多用户可能只使用 Art 站，主站归因覆盖率可能较低
3. **归因逻辑**：优先归因主站 → 如果用户有主站使用记录，优先归因到主站
4. **数据一致性验证**：总订单收入 = 主站归因 + Art归因 + 未归因
5. **成本单位**：energy 字段单位是 cents，需除以 100 转换为 USD

### FAQ

**Q1: 为什么主站归因收入可能比预期低？**
A: 因为很多付费用户可能主要使用 Art 站（图片生成），而主站功能（对话、聊天）使用较少。

**Q2: 如何理解"未归因"订单？**
A: 这些用户付费后从未使用任何功能（既没有主站记录也没有 Art 记录）。可能是购买后囤积能量、注册后未使用就退出、或数据延迟。

**Q3: 精确匹配的 ±5秒窗口是否足够？**
A: 经验证，±5秒窗口可以准确匹配大部分 Art 记录。如果发现匹配率异常，可以适当调整到 ±10秒。

**Q4: 如果用户同时使用主站和 Art 站，收入如何归因？**
A: 当前逻辑是优先归因主站。未来可以考虑按使用量比例分配。

### 相关文档

- `bot-margin-analysis.md` - Art 站 Bot 毛利率分析
- `main-site-energy-analysis.md` - 主站电量消耗分析
- `art-generation-country-analysis.ipynb` - Art 站国家分析

In [ ]:
// CLI 模式下退出进程，避免 MCP 连接保持进程不退出
if (Deno.args.length > 0) Deno.exit(0);

: 